In [102]:
import pandas as pd
import os

data_file = os.path.join('.', 'data', 'wlw_voice2text.csv')
data = pd.read_csv(data_file,encoding='utf-8',sep="#",header=0)
print(data)
print(data.columns.tolist())
print(data.shape)


          id                                               text
0  963838069  {"modelArray":[],"displayList":[{"Begin":5790,...
1  963838070  {"modelArray":[],"displayList":[{"Begin":2690,...
['id', 'text']
(2, 2)


In [103]:
data_id = data.iloc[:,0]
print(data_id)
data_json = data.iloc[:,1]
print(data_json)
type(data_json),data_json.name

0    963838069
1    963838070
Name: id, dtype: int64
0    {"modelArray":[],"displayList":[{"Begin":5790,...
1    {"modelArray":[],"displayList":[{"Begin":2690,...
Name: text, dtype: object


(pandas.core.series.Series, 'text')

In [104]:
import json

def safe_json_loads(json_str):
    try:
        # 尝试将字符串解析为 JSON 对象
        return json.loads(json_str)
    except json.JSONDecodeError:
        # 如果解析失败（例如，字符串不是有效的 JSON），返回 None 或者一个空字典，或者其他你希望的默认值
        print(f"Warning: Could not decode JSON string: {json_str}")
        return None # 或者 {} 或者 pd.NA

data_json = data_json.apply(safe_json_loads)
data_json


0    {'modelArray': [], 'displayList': [{'Begin': 5...
1    {'modelArray': [], 'displayList': [{'Begin': 2...
Name: text, dtype: object

In [105]:
data_json[0]['modelArray']
# data_json[0] = data_json[0]['displayList']
data_json.shape, type(data_json), type(data)

((2,), pandas.core.series.Series, pandas.core.frame.DataFrame)

In [106]:
def process_transcript_data(parsed_json_data):
    # 1. 检查输入是否为有效的字典
    if not isinstance(parsed_json_data, dict):
        return None # 如果不是字典，或者之前解析失败（为None），则返回None

    # 2. 获取 'displayList' 键的值
    # .get() 方法可以安全地访问字典中的键，如果键不存在则返回 None (或你指定的默认值)
    display_list = parsed_json_data.get('displayList')

    # 3. 检查 'displayList' 是否存在且为列表类型
    if not isinstance(display_list, list):
        return None # 如果 'displayList' 不存在或不是列表，则返回None

    # 4. 过滤并排序 displayList
    # 我们需要确保列表中的每个元素都是字典，并且包含 'Begin' 和 'Text' 键
    valid_segments = []
    for segment in display_list:
        if isinstance(segment, dict) and 'Begin' in segment and 'Text' in segment:
            valid_segments.append(segment)
        # else:
            # print(f"Warning: Skipping invalid segment in displayList: {segment}") # 可选：打印警告

    # 使用 sorted() 函数和 lambda 表达式按 'Begin' 字段升序排序
    # key=lambda x: x['Begin'] 表示以每个字典的 'Begin' 键的值作为排序依据
    try:
        sorted_segments = sorted(valid_segments, key=lambda x: x['Begin'])
    except TypeError:
        # 如果 'Begin' 字段的值不是可比较的类型（如数字），捕获 TypeError
        print(f"Warning: 'Begin' field in displayList contains non-comparable types.")
        return None
    except Exception as e:
        # 捕获其他可能的排序错误
        print(f"Error during sorting displayList: {e}")
        return None

    # 5. 提取 'Text' 字段并拼接成一个字符串
    # 使用列表推导式提取所有排好序的 'Text'
    # 使用 " ".join() 将它们用空格连接起来
    concatenated_text = ".".join([segment['Text'] for segment in sorted_segments])

    return concatenated_text


In [108]:
# print(process_transcript_data(data_json[0]))
data_json = pd.DataFrame(data_json)
type(data_json)
# data_json['text']
# data_json['text_res']=data_json['text'].apply(process_transcript_data)
# data_json['text_res'].head()

pandas.core.frame.DataFrame

In [109]:
data_json.columns,data_json.shape

(Index(['text'], dtype='object'), (2, 1))

In [110]:
data_json['text_res'] = data_json['text'].apply(process_transcript_data)
data_json.shape

(2, 2)

In [114]:
print(data_json['text_res'])

0    您好很高兴为您服务请问有什么可以帮您.就是我一个电话手表的卡正常用的然后也是在有效期现在突然...
1    你好很高兴为您服务请问有什么可以帮您.哎你好就是我今天.这种.刚才的时候客户反馈有20多张卡...
Name: text_res, dtype: object
